# Madrid Pet Bussiness

Establish a dog kindergarten service in the city of Madrid

It is planned to examine the city of Madrid, in order to establish a dog kennel; So, you need to see in which parts of the city are places with greater and lesser concentration related to pets, whether they are nurseries, vets, veterinary emergencies, parks.

So we are going to cluster the places in the city and see where we can be located with the greatest probability of success.

Madrid was chosen for being the city with the most business possibilities, in terms of the size of the market in Spain, as well as being the city of Spain with the most canine pets, compared to other cities, both percentage-wise at the level of the general population. Madrid , as in number of canine pets.



### Why is this data useful for my business?

The general idea is to bring the relevant data to examine on the map of Madrid, which areas have a better environment to care for dogs.
Basically we will see that areas or neighborhoods of the city have more parks and services for pets.

The data of the neighborhoods and areas of Madrid were taken from the official data page of the Spanish government: https://datos.gob.es/es/catalogo/a13002908-barrios-del-munbierno-de-madrid

And the data of the places analyzed was taken from the foursquare.com API.



In [1]:
# import the libraries

import urllib.request
import pandas as pd
import numpy as np
from sklearn import preprocessing
import random
import requests
from pandas.io.json import json_normalize 
import json

### Download and Explore 'Barrios' of Madrid

In [2]:
# Dowloading data

#!wget -q -O 'barrios_municipio_madrid.csv' https://datos.comunidad.madrid/catalogo/dataset/91857345-056b-4aa7-96f5-6310dac619bf/resource/714b237e-2039-4bcd-ad3d-45408d315ed4/download/barrios_municipio_madrid.csv
#print('Data downloaded!')

In [3]:
# Data to dataframe

barriosmadrid = pd.read_csv('barrios_municipio_madrid.csv', encoding='latin-1', sep=";") #error_bad_lines=False, 
barriosmadrid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   barrio_codigo     128 non-null    int64  
 1   barrio_nombre     128 non-null    object 
 2   distrito_codigo   128 non-null    int64  
 3   distrito_nombre   128 non-null    object 
 4   municipio_codigo  128 non-null    int64  
 5   municipio_nombre  128 non-null    object 
 6   superficie_km2    128 non-null    float64
 7   densidad_por_km2  128 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 8.1+ KB


In [4]:
barriosmadrid.head()

,barrio_codigo,barrio_nombre,distrito_codigo,distrito_nombre,municipio_codigo,municipio_nombre,superficie_km2,densidad_por_km2
0,796011,Palacio,79601,Centro,796,Madrid,1.46,15323.29
1,796012,Embajadores,79601,Centro,796,Madrid,1.03,43345.63
2,796013,Cortes,79601,Centro,796,Madrid,0.59,17850.85
3,796014,Justicia,79601,Centro,796,Madrid,0.75,21866.67
4,796015,Universidad,79601,Centro,796,Madrid,0.93,33051.61


### Get more data about Madrid

In [5]:
# Use geopy library to get the latitude and longitude values of Madrid City.

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  openssl                                 1.1.1g-h516909a_0 --> 1.1.1g-h516909a_1



openssl-1.1.1g       | 2.1 MB    | #####

In [119]:
address = 'Madrid, ES'

geolocator = Nominatim(user_agent="md_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid are 40.4167047, -3.7035825.


In [120]:
# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=10)
   
map_madrid

### Go to Foursquare to get venues, 'Parques' of Madrid 

In [121]:
CLIENT_ID = 'QD2AGHVK4EGBFXSQ2SHYQPX0ARDPZ15BCELOFHFKKE03VCTJ' # your Foursquare ID
CLIENT_SECRET = 'SRHMFBI2R3JFND0FB15UDGD3SGVVXS1F3IIIHCJDPUZAYHAY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QD2AGHVK4EGBFXSQ2SHYQPX0ARDPZ15BCELOFHFKKE03VCTJ
CLIENT_SECRET:SRHMFBI2R3JFND0FB15UDGD3SGVVXS1F3IIIHCJDPUZAYHAY


In [123]:
# Define the URL

# https://api.foursquare.com/v2/venues/search?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&query=QUERY&radius=RADIUS&limit=LIMIT

search_query = 'Parque'
radius = 10000
LIMIT = 3000

In [124]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=QD2AGHVK4EGBFXSQ2SHYQPX0ARDPZ15BCELOFHFKKE03VCTJ&client_secret=SRHMFBI2R3JFND0FB15UDGD3SGVVXS1F3IIIHCJDPUZAYHAY&ll=40.4167047,-3.7035825&v=20180605&query=Parque&radius=10000&limit=3000'

In [125]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2c1429e70bb116fe5c4928'},
 'response': {'venues': [{'id': '4b8aa4c2f964a5201d7732e3',
    'name': 'Parque del Retiro',
    'location': {'address': 'Pl. de la Independencia, 7',
     'crossStreet': 'C. Alcalá',
     'lat': 40.41517879765204,
     'lng': -3.6814713478088374,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.41517879765204,
       'lng': -3.6814713478088374}],
     'distance': 1881,
     'postalCode': '28014',
     'cc': 'ES',
     'city': 'Madrid',
     'state': 'Madrid',
     'country': 'España',
     'formattedAddress': ['Pl. de la Independencia, 7 (C. Alcalá)',
      '28014 Madrid Madrid',
      'España']},
    'categories': [{'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'shortName': 'Park',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1596724321',
    'hasPer

### Let go to see that parks

In [128]:
venues = results['response']['venues']
    
venues = pd.json_normalize(venues) # flatten JSON

venues.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4b8aa4c2f964a5201d7732e3,Parque del Retiro,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1596724321,False,"Pl. de la Independencia, 7",C. Alcalá,40.415179,-3.681471,"[{'label': 'display', 'lat': 40.41517879765204...",1881,28014,ES,Madrid,Madrid,España,"[Pl. de la Independencia, 7 (C. Alcalá), 28014...",NaN
1,4f6f520be4b056aa9465342a,Parque Madrid Río (Bosque de Trepa),"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1596724321,False,NaN,NaN,40.420484,-3.704736,"[{'label': 'display', 'lat': 40.42048401468789...",431,28011,ES,Madrid,Madrid,España,"[28011 Madrid Madrid, España]",NaN
2,4bdc5babfed22d7f92ca57c9,Parque del Oeste,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1596724321,False,"P. del Pintor Rosales, s/n",NaN,40.428123,-3.721987,"[{'label': 'display', 'lat': 40.428123, 'lng':...",2012,28008,ES,Madrid,Madrid,España,"[P. del Pintor Rosales, s/n, 28008 Madrid Madr...",NaN
3,50d86791e4b07ec189f70cb3,Parque Infantil Plaza De Oriente,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1596724321,False,NaN,NaN,40.417535,-3.712035,"[{'label': 'display', 'lat': 40.417535, 'lng':...",722,NaN,ES,NaN,NaN,España,[España],NaN
4,4e3bc1b1b0fb875af84a53ae,Parque infantil,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",v-1596724321,False,NaN,NaN,40.417478,-3.712095,"[{'label': 'display', 'lat': 40.41747776, 'lng...",726,NaN,ES,Madrid,Madrid,España,"[Madrid Madrid, España]",NaN


### Maybe it is better if we see them in a Madrid map

In [129]:
lat = venues['location.lat']
lng = venues['location.lng']

# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(venues['location.lat'], venues['location.lng'], venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

### Where are we go to establish our bussiness? 